In [5]:
import numpy as np
import pandas as pd
import keras

from timeit import default_timer
from sklearn.model_selection import train_test_split
from keras.layers import Activation
from keras.layers import Conv2D, BatchNormalization, Dense, Flatten, Reshape
import tensorflow as tf
from google.colab import userdata
import os
!pip install keras --upgrade
!pip install datasets
os.environ["KERAS_BACKEND"] = "torch"
from keras import models, callbacks, utils
from datasets import Dataset
import numpy as np
from tqdm.auto import trange

from keras import models, utils
from huggingface_hub import hf_hub_download
from tensorflow import keras
import tensorflow as tf
from tensorflow.python.keras import layers
from tensorflow.python.keras.models import Sequential
from datasets import load_dataset
import numpy as np
from sklearn.model_selection import train_test_split

In [3]:
"""
Please uncomment those lines if you are running this code on Google Colab
Else if you are running this code on your local machine, please use this dataset 
https://www.kaggle.com/datasets/radcliffe/3-million-sudoku-puzzles-with-ratings

from google.colab import userdata
import os

!pip install kaggle
os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')
!kaggle datasets download -d radcliffe/3-million-sudoku-puzzles-with-ratings
!unzip 3-million-sudoku-puzzles-with-ratings.zip """

 94% 195M/207M [00:02<00:00, 99.6MB/s]
100% 207M/207M [00:02<00:00, 75.7MB/s]
Archive:  3-million-sudoku-puzzles-with-ratings.zip
  inflating: sudoku-3m.csv           


### Get the data and pre processed it.

In [52]:
# Read the dataset
data = pd.read_csv("sudoku-3m.csv", skiprows=lambda i: i % 10 != 0)

In [54]:
print(data.shape) # Display the number of samples and features

(300000, 5)


In [55]:
def preprocess_data(data):
    # Replace periods with zeros in puzzle strings
    data['puzzle'] = data['puzzle'].str.replace('.', '0')

    # Extract Sudoku grid values (first 81 characters of the puzzle string)
    X = np.array([list(map(int, puzzle[:81])) for puzzle in data['puzzle']])
    print(X)
    y = data['solution'].values  # No need to adjust targets here

    # Reshape X to have dimensions (num_samples, 9, 9, 1)
    X = X.reshape(-1, 9, 9, 1)

    # Normalize input
    X = X / 9.0  # Sudoku values range from 1 to 9

    return X, y


# Split dataset into train and test sets
X, y = preprocess_data(data)

[[0 7 5 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [2 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 9 0 0]
 [2 0 0 ... 0 4 0]
 [7 0 4 ... 0 4 7]]


In [56]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = np.eye(10)[X_train.reshape(-1).astype(int)].reshape(X_train.shape[0], 9, 9, 10)
X_test = np.eye(10)[X_test.reshape(-1).astype(int)].reshape(X_test.shape[0], 9, 9, 10)
y_train = np.array([list(map(int, y)) for y in y_train]).reshape(y_train.shape[0], -1) - 1
y_test = np.array([list(map(int, y)) for y in y_test]).reshape(y_test.shape[0], -1) - 1

In [60]:
!nvidia-smi # Display the GPU information, be sure to run this code if you are on Google Colab

Mon Apr  8 17:11:54 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   76C    P0              32W /  70W |   2503MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

### Now that we have our data and pre processed it, we can define our model and train it

In [61]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, models

# Define the model architecture
def build_model():
    model_input = layers.Input(shape=(9, 9, 10), name="puzzle")
    x = layers.Dense(64, activation="relu", name="block_01_linear")(model_input)
    x = layers.Dropout(0.4, name="block_01_dopout")(x)
    x = layers.Dense(64, activation="relu", name="block_02_linear")(x)
    x = layers.Dropout(0.4, name="block_02_dropout")(x)
    x = layers.Flatten(name="flatten")(x)
    model_output = [
        layers.Dense(9, activation="softmax", name=f"position_{i+1}_{j+1}")(x) for i in range(9) for j in range(9)
    ]

    model = models.Model(
        inputs=model_input,
        outputs=[model_output[i * 9 + j] for i in range(9) for j in range(9)],
        name="SuDoKuNetFFN",
    )
    return model

# Build the model
model = build_model()

model.compile(
    optimizer="adam", # Optimizer
    loss=["categorical_crossentropy"]*81,
    metrics=[["accuracy"]]*81
)

model.fit(
    X_train,
    [y_train[:, i*9:(i+1)*9] for i in range(9)],
    epochs=50, # Number of epochs
    batch_size=81, # Number of samples per gradient update
    validation_split=0.1 # Fraction of the training data to be used as validation data
)

score = model.evaluate(
    X_test,
    [y_test[:, i*9:(i+1)*9] for i in range(9)]
)



Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/optimizers/base_optimizer.py:576: UserWarning: Gradients do not exist for variables ['kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', 'kernel', 'bias', '

2667/2667 ━━━━━━━━━━━━━━━━━━━━ 49s 12ms/step - loss: 209315.3125 - position_1_1_accuracy: 0.1125 - position_1_2_accuracy: 0.1115 - position_1_3_accuracy: 0.1131 - position_1_4_accuracy: 0.1095 - position_1_5_accuracy: 0.1119 - position_1_6_accuracy: 0.1103 - position_1_7_accuracy: 0.1120 - position_1_8_accuracy: 0.1130 - position_1_9_accuracy: 0.1113 - val_loss: 988767.5625 - val_position_1_1_accuracy: 0.1108 - val_position_1_2_accuracy: 0.1129 - val_position_1_3_accuracy: 0.1140 - val_position_1_4_accuracy: 0.1120 - val_position_1_5_accuracy: 0.1101 - val_position_1_6_accuracy: 0.1132 - val_position_1_7_accuracy: 0.1130 - val_position_1_8_accuracy: 0.1104 - val_position_1_9_accuracy: 0.1148
Epoch 2/50
2667/2667 ━━━━━━━━━━━━━━━━━━━━ 22s 7ms/step - loss: 2293447.5000 - position_1_1_accuracy: 0.1101 - position_1_2_accuracy: 0.1093 - position_1_3_accuracy: 0.1125 - position_1_4_accuracy: 0.1109 - position_1_5_accuracy: 0.1099 - position_1_6_accuracy: 0.1106 - position_1_7_accuracy: 0.1131

In [63]:
model.save('ffn_model.keras') # Save the model

In [42]:
# Simple test function 
def test_solver():
    # Define some test cases
    test_cases = [
        {
            "puzzle": "530070000600195000098000060800060003400803001700020006060000280000419005000080079",
            "solution": "534678912672195348198342567859761423426853791713924856961537284287419635345286179"
        },
        {
            "puzzle": "000000000000003085001020000000507000004000100090000000500000073002010000000040009",
            "solution": "987654321246173985351928746128537694634892157795461832519286473472319568863745219"
        },
        {
            "puzzle": "000000000000000000000000000000000000000000000000000000000000000000000000000000000",
            "solution": "123456789456789123789123456214365897365897214897214365531642978642978531978531642"
        }
    ]

    # Iterate over test cases
    for i, case in enumerate(test_cases):
        print(f"Test Case #{i+1}:")

        # Prepare input data
        test_input = np.array(list(map(int, case['puzzle']))).reshape(1, 9, 9, 1)
        test_input = np.eye(10)[test_input.reshape(-1).astype(int)].reshape(test_input.shape[0], 9, 9, 10)

        # Get model's prediction
        prediction = model.predict(test_input)
        predicted_solution = "".join(str(np.argmax(pos)+1) for pos_row in prediction for pos in pos_row)

        # Compare with expected solution
        expected_solution = case['solution']
        if predicted_solution == expected_solution:
            print("Passed!")
        else:
            print(f"Failed. Expected {expected_solution}, but got {predicted_solution}")

        print()

# Run the tests
test_solver()

Test Case #1:
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
Failed. Expected 534678912672195348198342567859761423426853791713924856961537284287419635345286179, but got 442886912625674682942434684693424316516911954971218646137386522685752866655756352

Test Case #2:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
Failed. Expected 987654321246173985351928746128537694634892157795461832519286473472319568863745219, but got 462588919925674992942334944918314316216511954431268684137586422545752866655732352

Test Case #3:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Failed. Expected 123456789456789123789123456214365897365897214897214365531642978642978531978531642, but got 472983917915674692945334944298344316516511954837268684747886522545752866655752352

